# Assignment 3: Probabilistic modeling

In this assignment, you will work with probabilistic models known as Bayesian networks to efficiently calculate the answer to probability questions concerning discrete random variables.

To help, we've provided a package called [pbnt](https://github.com/achille/pbnt) that supports the representation of Bayesian networks and automatic inference of marginal probabilities. Note that you need numpy to run pbnt.

This assignment is due on T-Square by Feb 26th, 11:59PM UTC-12. Please submit your solution as probability_solution.py.

**Warning**
-----

Due to compatibility bugs in pbnt, **this assignment requires Python 2.7 to run, which in turn requires iPython2.** So you should run this notebook using

    ipython2 notebook probability_notebook.ipynb

If you don't have iPython2 installed, you can download it [here](https://github.com/ipython/ipython/archive/rel-2.4.1.zip), unzip it, and install it using

    python setup.py install

If you don't have iPython2 installed and you don't want to have more than one version installed, you can set it up using a virtual environment (virtualenv). 

You can find instructions on how to set up a virtualenv under the following links:

- [Windows users](http://www.tylerbutler.com/2012/05/how-to-install-python-pip-and-virtualenv-on-windows-with-powershell/) (TL;DR use PowerShell)
- [Linux/Mac users](https://virtualenv.pypa.io/en/latest/installation.html) (TL;DR use pip if you can)

Once you have virtualenv installed, you should navigate to the directory containing probability_notebook.ipynb and run the command

    virtualenv .
    
which will create a subdirectory called "bin" which contains scripts to run the virtual environment. You'll then call

    source bin/activate
    
to activate the virtualenv. You'll see that your command line now looks something like this:

    (assignment_3)my-laptop:probability_assignment user_name$

From here, you can install iPython2 to your local directory by running the command

    pip2.7 install "ipython [notebook]"
    
and then you'll be able to open probability_notebook.ipynb using iPython2.

Whenever you want to quit your virtual environment, just type

    deactivate
    
and you can reactivate the environment later with the same command you used before. If you ever want to get rid of the virtualenv files entirely, just delete the "bin" folder.

In [1]:
"""Testing pbnt. Run this before anything else to get pbnt to work!"""
import sys
if('pbnt/combined' not in sys.path):
    sys.path.append('pbnt/combined')
from exampleinference import inferenceExample

inferenceExample()
# Should output:
# ('The marginal probability of sprinkler=false:', 0.80102921)
#('The marginal probability of wetgrass=false | cloudy=False, rain=True:', 0.055)

('The marginal probability of sprinkler=false:', 0.80102921)
('The marginal probability of wetgrass=false | cloudy=False, rain=True:', 0.055)


Part 1: Bayesian network tutorial
-------
35 points total

To start, design a basic probabilistic model for the following system:

There's a nuclear power plant in which an alarm is supposed to ring when the core temperature, indicated by a gauge, exceeds a fixed threshold. For simplicity, we assume that the temperature is represented as either high or normal. However, the alarm is sometimes faulty, and the gauge is more likely to fail when the temperature is high. Use the following Boolean variables in your implementation:

- A = alarm sounds
- F<sub>A</sub> = alarm is faulty
- G = gauge reading (high = True, normal = False)
- F<sub>G</sub> = gauge is faulty
- T = actual temperature (high = True, normal = False)

You will test your implementation at the end of the section.

1a: Casting the net
--
10 points

Design a Bayesian network for this system, using pbnt to represent the nodes and conditional probability arcs connecting nodes. Don't worry about the probabilities for now. Fill out the function below to create the net.

The following command will create a BayesNode with 2 values, an id of 0 and the name "alarm":

    A_node = BayesNode(0,2,name='alarm')

NOTE: Do not use any special characters(like $,_,-) for the name parameter.

You will use BayesNode.add\_parent() and BayesNode.add\_child() to connect nodes. For example, to connect the alarm and temperature nodes that you've already made (i.e. assuming that temperature affects the alarm probability):

    A_node.add_parent(T_node)
    T_node.add_child(A_node)
    
You can run probability\_tests.network\_setup\_test() to make sure your network is set up correctly.

       Hint : Checkout ExampleModels.py under pbnt/combined 

In [2]:
from Node import BayesNode
from Graph import BayesNet
def make_power_plant_net():
    """Create a Bayes Net representation of the above power plant problem. 
    Use the following as the name attribute: "alarm","faulty alarm", "gauge","faulty gauge", "temperature". (for the tests to work.)
    """
    nodes = []
    # TODO: finish this function
    T_node = BayesNode(0, 2, name='temperature')
    G_node = BayesNode(1, 2, name='gauge')
    F_G_node = BayesNode(2, 2, name='faulty gauge')
    F_A_node = BayesNode(3, 2, name='faulty alarm')
    A_node = BayesNode(4, 2, name='alarm')
    
    T_node.add_child(G_node)
    T_node.add_child(F_G_node)
    
    F_G_node.add_parent(T_node)
    F_G_node.add_child(G_node)
    
    G_node.add_parent(T_node)
    G_node.add_parent(F_G_node)
    G_node.add_child(A_node)
    
    F_A_node.add_child(A_node)
    
    A_node.add_parent(G_node)
    A_node.add_parent(F_A_node)
    
    nodes = [T_node, G_node, F_G_node, F_A_node, A_node]
    
    return BayesNet(nodes)

In [3]:
from probability_tests import network_setup_test
power_plant = make_power_plant_net()
network_setup_test(power_plant)

correct number of nodes
correct number of edges between nodes


1b: Setting the probabilities
---
15 points

Assume that the following statements about the system are true:

1. The temperature gauge reads the correct temperature with 95% probability when it is not faulty and 20% probability when it is faulty. For simplicity, say that the gauge's "true" value corresponds with its "hot" reading and "false" with its "normal" reading, so the gauge would have a 95% chance of returning "true" when the temperature is hot and it is not faulty.
2. The alarm is faulty 15% of the time.
3. The temperature is hot (call this "true") 20% of the time.
4. When the temperature is hot, the gauge is faulty 80% of the time. Otherwise, the gauge is faulty 5% of the time.
5. The alarm responds correctly to the gauge 55% of the time when the alarm is faulty, and it responds correctly to the gauge 90% of the time when the alarm is not faulty. For instance, when it is faulty, the alarm sounds 55% of the time that the gauge is "hot" and remains silent 55% of the time that the gauge is "normal."

Knowing these facts, set the conditional probabilities for the necessary variables on the network you just built.

Using pbnt's Distribution class: if you wanted to set the distribution for P(A) to 70% true, 30% false, you would invoke the following commands:

    A_distribution = DiscreteDistribution(A_node)
    index = A_distribution.generate_index([],[])
    A_distribution[index] = [0.3,0.7]
    A_node.set_dist(A_distribution)

REMEMBER: USE INDEX 0 FOR FALSE IN THE CODE.

If you wanted to set the distribution for P(A|G) to be

|$G$|$P(A=true| G)$|
|------|-----|
|T| 0.75|
|F| 0.85| 

you would invoke:

    from numpy import zeros, float32
    dist = zeros([G_node.size(), A_node.size()], dtype=float32)   #Note the order of G_node, A_node
    dist[0,:] = [0.15, 0.85]
    dist[1,:] = [0.25, 0.75]
    A_distribution = ConditionalDiscreteDistribution(nodes=[G_node,A_node], table=dist)
    A_node.set_dist(A_distribution)

Modeling a three-variable relationship is a bit trickier. If you wanted to set the following distribution for $P(A|G,T)$ to be

|$G$|$T$|$P(A=true| G, T)$|
|--|--|:----:|
|T|T|0.15|
|T|F|0.6|
|F|T|0.2|
|F|F|0.1|

you would invoke:

    from numpy import zeros, float32
    dist = zeros([G_node.size(), T_node.size(), A_node.size()], dtype=float32)
    dist[0,0,:] = [0.9, 0.1]
    dist[0,1,:] = [0.8, 0.2]
    dist[1,0,:] = [0.4, 0.6]
    dist[1,1,:] = [0.85, 0.15]
    
    A_distribution = ConditionalDiscreteDistribution(nodes=[G_node, T_node, A_node], table=dist)
    A_node.set_dist(A_distribution)

The key is to remember that 0 represents the index of the false probability, and 1 represents true.

You can check your probability distributions with probability_tests.probability_setup_test().

In [4]:
# from numpy import zeros, float32
import numpy as np
import Distribution
from Distribution import DiscreteDistribution, ConditionalDiscreteDistribution
def set_probability(bayes_net):
    """Set probability distribution for each node in the power plant system."""
    
    A_node = bayes_net.get_node_by_name("alarm")
    F_A_node = bayes_net.get_node_by_name("faulty alarm")
    G_node = bayes_net.get_node_by_name("gauge")
    F_G_node = bayes_net.get_node_by_name("faulty gauge")
    T_node = bayes_net.get_node_by_name("temperature")
    nodes = [A_node, F_A_node, G_node, F_G_node, T_node]
    # TODO: set the probability distribution for each node

    #1
    dist = np.zeros([F_G_node.size(), T_node.size(), G_node.size()], dtype=np.float32)
    dist[0, 0, :] = [0.05, 0.95]
    dist[0, 1, :] = [0.05, 0.95]
    dist[1, 0, :] = [0.8, 0.2]
    dist[1, 1, :] = [0.8, 0.2]
    G_distribution = ConditionalDiscreteDistribution(nodes=[F_G_node, T_node, G_node], table=dist)
    G_node.set_dist(G_distribution)
    
    #2
    F_A_distribution = DiscreteDistribution(F_A_node)
    index = F_A_distribution.generate_index([], [])
    F_A_distribution[index] = [0.85, 0.15]
    F_A_node.set_dist(F_A_distribution)
    
    #3
    T_distribution = DiscreteDistribution(T_node)
    index = T_distribution.generate_index([], [])
    T_distribution[index] = [0.8, 0.2]
    T_node.set_dist(T_distribution)
    
    #4
    dist = np.zeros([T_node.size(), F_G_node.size()], dtype=np.float32)
    dist[0, :] = [0.95, 0.05]
    dist[1, :] = [0.2, 0.8]
    F_G_distribution = ConditionalDiscreteDistribution(nodes=[T_node, F_G_node], table=dist)
    F_G_node.set_dist(F_G_distribution)
    
    #5
    dist = np.zeros([F_A_node.size(), G_node.size(), A_node.size()], dtype=np.float32)
    dist[0, 0, :] = [0.1, 0.9]
    dist[0, 1, :] = [0.1, 0.9]
    dist[1, 0, :] = [0.45, 0.55]
    dist[1, 1, :] = [0.45, 0.55]
    A_distribution = ConditionalDiscreteDistribution(nodes=[F_A_node, G_node, A_node], table=dist)
    A_node.set_dist(A_distribution)
    
    return bayes_net

In [5]:
set_probability(power_plant)
from probability_tests import probability_setup_test
probability_setup_test(power_plant)

checking probability distribution for Temperature node...
correct temperature distribution size
correct temperature distribution
checking probability distribution for Faulty Gauge node...
correct faulty gauge distribution size
correct faulty gauge distribution
checking probability distribution for Faulty Alarm node...
correct faulty alarm distribution size
correct faulty alarm distribution
checking only the probability distribution size for Gauge node...
correct gauge distribution size
checking only the probability distribution size for Alarm node...
correct alarm distribution size


1c: Probability calculations : Perform inference
---
10 points

To finish up, you're going to perform inference on the network to calculate the following probabilities:

- the marginal probability that the alarm sounds
- the marginal probability that the gauge shows "hot"
- the probability that the temperature is actually hot, given that the alarm sounds and the alarm and gauge are both working

You'll fill out the "get_prob" functions to calculate the probabilities.

Here's an example of how to do inference for the marginal probability of the "faulty alarm" node being True (assuming "bayes_net" is your network):

    F_A_node = bayes_net.get_node_by_name('faulty alarm')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(F_A_node)[0]
    index = Q.generate_index([True],range(Q.nDims))
    prob = Q[index]
  
To compute the conditional probability, set the evidence variables before computing the marginal as seen below (here we're computing $P(A = false | F_A = true, T = False)$):

    engine.evidence[F_A_node] = True
    engine.evidence[T_node] = False
    Q = engine.marginal(A_node)[0]
    index = Q.generate_index([False],range(Q.nDims))
    prob = Q[index]

If you need to sanity-check to make sure you're doing inference correctly, you can run inference on one of the probabilities that we gave you in 1b. For instance, running inference on $P(T=true)$ should return 0.19999994 (i.e. almost 20%). You can also calculate the answers by hand to double-check.

In [6]:
from Inference import JunctionTreeEngine
def get_alarm_prob(bayes_net, alarm_rings):
    """Calculate the marginal 
    probability of the alarm 
    ringing (T/F) in the 
    power plant system."""
    # TODO: finish this function
    A_node = bayes_net.get_node_by_name('alarm')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(A_node)[0]
    index = Q.generate_index([alarm_rings], range(Q.nDims))
    alarm_prob = Q[index]
    return alarm_prob

In [7]:
def get_gauge_prob(bayes_net, gauge_hot):
    """Calculate the marginal
    probability of the gauge 
    showing hot (T/F) in the 
    power plant system."""
    # TOOD: finish this function
    G_node = bayes_net.get_node_by_name('gauge')
    engine = JunctionTreeEngine(bayes_net)
    Q = engine.marginal(G_node)[0]
    index = Q.generate_index([gauge_hot], range(Q.nDims))
    gauge_prob = Q[index]
    return gauge_prob

In [8]:
def get_temperature_prob(bayes_net,temp_hot):
    """Calculate the conditional probability 
    of the temperature being hot (T/F) in the
    power plant system, given that the
    alarm sounds and neither the gauge
    nor alarm is faulty."""
    # TODO: finish this function
    T_node = bayes_net.get_node_by_name('temperature')
    A_node = bayes_net.get_node_by_name('alarm')
    F_A_node = bayes_net.get_node_by_name('faulty alarm')
    F_G_node = bayes_net.get_node_by_name('faulty gauge')
    engine = JunctionTreeEngine(bayes_net)
    engine.evidence[A_node] = True
    engine.evidence[F_A_node] = False
    engine.evidence[F_G_node] = False
    Q = engine.marginal(T_node)[0]
    index = Q.generate_index([temp_hot], range(Q.nDims))
    temp_prob = Q[index]
    return temp_prob

In [9]:
print get_alarm_prob(power_plant,True)
print get_gauge_prob(power_plant,True)
print get_temperature_prob(power_plant,True)

0.8475
0.8
0.05


Part 2: Sampling
-----
65 points total

For the main exercise, consider the following scenario.

There are three frisbee teams who play each other: the Airheads, the Buffoons, and the Clods (A, B and C for short). 
Each match is between two teams, and each team can either win, lose, or draw in a match. Each team has a fixed but 
unknown skill level, represented as an integer from 0 to 3. The outcome of each match is probabilistically proportional to the difference in skill level between the teams.

Sampling is a method for ESTIMATING a probability distribution when it is prohibitively expensive (even for inference!) to completely compute the distribution. 

Here, we want to estimate the outcome of the matches, given prior knowledge of previous matches. Rather than using inference, we will do so by sampling the network using two [Markov Chain Monte Carlo](http://www.statistics.com/papers/LESSON1_Notes_MCMC.pdf) models: Gibbs sampling (2c) and Metropolis-Hastings (2d).

2a: Build the network.
-----
10 points

For the first sub-part, consider a network with 3 teams : the Airheads, the Buffoons, and the Clods (A, B and C for short). 3 total matches are played. 
Build a Bayes Net to represent the three teams and their influences on the match outcomes. Assume the following variable conventions:

| variable name | description|
|---------|:------:|
|A| A's skill level|
|B | B's skill level|
|C | C's skill level|
|AvB | the outcome of A vs. B <br> (0 = A wins, 1 = B wins, 2 = tie)|
|BvC | the outcome of B vs. C <br> (0 = B wins, 1 = C wins, 2 = tie)|
|CvA | the outcome of C vs. A <br> (0 = C wins, 1 = A wins, 2 = tie)|

Assume that each team has the following prior distribution of skill levels:

|skill level|P(skill level)|
|----|:----:|
|0|0.15|
|1|0.45|
|2|0.30|
|3|0.10|

In addition, assume that the differences in skill levels correspond to the following probabilities of winning:

| skill difference <br> (T2 - T1) | T1 wins | T2 wins| Tie |
|------------|----------|---|:--------:|
|0|0.10|0.10|0.80|
|1|0.20|0.60|0.20|
|2|0.15|0.75|0.10|
|3|0.05|0.90|0.05|

In [10]:
def set_skill_distribution(node):
    distribution = DiscreteDistribution(node)
    index = distribution.generate_index([], [])
    distribution[index] = [0.15, 0.45, 0.3, 0.1]
    node.set_dist(distribution)
    return node


def set_skill_diff_distribution(node, dist_nodes):
    dist = np.zeros([dist_node.size() for dist_node in dist_nodes], dtype=np.float32)
    # skill diff = 0
    dist[0, 0] = dist[1, 1] = dist[2, 2] = dist[3, 3] = [0.1, 0.1, 0.8]

    # skill diff = 1
    dist[0, 1] = dist[1, 2] = dist[2, 3] = [0.2, 0.6, 0.2]
    dist[1, 0] = dist[2, 1] = dist[3, 2] = [0.6, 0.2, 0.2]

    # skill diff = 2
    dist[0, 2] = dist[1, 3] = [0.15, 0.75, 0.1]
    dist[2, 0] = dist[3, 1] = [0.75, 0.15, 0.1]

    # skill diff = 3
    dist[0, 3] = [0.05, 0.9, 0.05]
    dist[3, 0] = [0.9, 0.05, 0.05]
    
    distribution = ConditionalDiscreteDistribution(nodes=dist_nodes, table=dist)
    node.set_dist(distribution)
    return node


def get_game_network():
    """Create a Bayes Net representation of the game problem.
    Name the nodes as "A","B","C","AvB","BvC" and "CvA".  """
    nodes = []
    # TODO: fill this out
    A_node = BayesNode(0, 4, name='A')
    B_node = BayesNode(1, 4, name='B')
    C_node = BayesNode(2, 4, name='C')
    AvB_node = BayesNode(3, 3, name='AvB')
    BvC_node = BayesNode(4, 3, name='BvC')
    CvA_node = BayesNode(5, 3, name='CvA')
    
    A_node.add_child(AvB_node)
    A_node.add_child(CvA_node)
    
    B_node.add_child(AvB_node)
    B_node.add_child(BvC_node)
    
    C_node.add_child(BvC_node)
    C_node.add_child(CvA_node)
    
    AvB_node.add_parent(A_node)
    AvB_node.add_parent(B_node)
    
    BvC_node.add_parent(B_node)
    BvC_node.add_parent(C_node)
    
    CvA_node.add_parent(A_node)
    CvA_node.add_parent(C_node)
    
    nodes = [A_node, B_node, C_node, AvB_node, BvC_node, CvA_node]
    
    A_node = set_skill_distribution(A_node)
    B_node = set_skill_distribution(B_node)
    C_node = set_skill_distribution(C_node)
    
    AvB_node = set_skill_diff_distribution(AvB_node, [A_node, B_node, AvB_node])
    BvC_node = set_skill_diff_distribution(BvC_node, [B_node, C_node, BvC_node])
    CvA_node = set_skill_diff_distribution(CvA_node, [C_node, A_node, CvA_node])

    return BayesNet(nodes)

In [11]:
game_net = get_game_network()

from probability_tests import games_network_test
games_network_test(game_net)

checking the total number of edges and nodes in the network...
correct number of nodes
correct number of edges
checking probability distribution for Team A...
correct team distribution size
correct team distribution
checking probability distribution for match AvB...
correct match distribution size
correct match distribution


2b: Calculate posterior distribution for the 3rd match.
----
5 points

Suppose that you know the following outcome of two of the three games: A beats B and A draws with C. Calculate the posterior distribution for the outcome of the BvC match in calculate_posterior(). Use EnumerationEngine ONLY. 

In [12]:
from Inference import EnumerationEngine

def calculate_posterior(bayes_net):
    """Calculate the posterior distribution of the BvC match given that A won against B and tied C. 
    Return a list of probabilities corresponding to win, loss and tie likelihood."""
    posterior = [0,0,0]
    # TODO: finish this function
    AvB_node = bayes_net.get_node_by_name('AvB')
    CvA_node = bayes_net.get_node_by_name('CvA')
    BvC_node = bayes_net.get_node_by_name('BvC')
    
    engine = EnumerationEngine(bayes_net)
    engine.evidence[AvB_node] = 0
    engine.evidence[CvA_node] = 2
    Q = engine.marginal(BvC_node)[0]
    
    match_results = [0, 1, 2]
    for i, result in enumerate(match_results):
        index = Q.generate_index([result], range(Q.nDims))
        posterior[i] = Q[index]
    
    return posterior # list

In [13]:
from probability_tests import posterior_test
posterior = calculate_posterior(game_net)
posterior_test(posterior)
print posterior

correct posterior
[0.25890073, 0.42796761, 0.3131316]


2c: Gibbs sampling
---
15 points

Implement the Gibbs sampling algorithm, which is a special case of Metropolis-Hastings. You'll do this in Gibbs_sampler(), which takes a Bayesian network and initial state value as a parameter and returns a sample state drawn from the network's distribution. In case of Gibbs, the returned state differs from the input state at at-most one variable (randomly chosen).

The method should just consist of a single iteration of the algorithm. If an initial value is not given, default to a state chosen uniformly at random from the possible states.

Note: DO NOT USE the given inference engines to run the sampling method, since the whole point of sampling is to calculate marginals without running inference. 


     "YOU WILL SCORE 0 POINTS ON THIS ASSIGNMENT IF YOU USE THE GIVEN INFERENCE ENGINES FOR THIS PART!!"


You may find [this](http://gandalf.psych.umn.edu/users/schrater/schrater_lab/courses/AI2/gibbs.pdf) helpful in understanding the basics of Gibbs sampling over Bayesian networks. 

Hint 1: In both Metropolis-Hastings and Gibbs sampling, you'll need access to each node's probability distribution and nodes. 
You can access these by calling: 
    
    A = bayes_net.get_node_by_name("A")      
    team_table = A.dist.table
    AvB = bayes_net.get_node_by_name("AvB")
    match_table = AvB.dist.table
    
which will return the same numpy array that you provided when constructing the probability distribution.

Hint 2: you'll also want to use the random package (e.g. random.randint()) for the probabilistic choices that sampling makes.

Hint 3: in order to count the sample states later on, you'll want to make sure the sample that you return is hashable. One way to do this is by returning the sample as a tuple.


In [14]:
def get_random_state(nodes):
    return [np.random.choice(xrange(node.numValues)) for node in nodes]


def get_initial_state(nodes, initial_state):
    # if initial_state is not defined, get random initial
    if initial_state is None or len(initial_state) != len(nodes):
        return get_random_state(nodes)
    return initial_state


def get_markov_blanket_probability(nodes, node, state):
    # markov blanket includes its parent, children, and the other parents
    p = get_probability(nodes, node, state)
    for child in node.children:
        p = p * get_probability(nodes, child, state)
    return p


def get_probability(nodes, node, state):
    dist = node.dist.table
    # has parents means node is match
    if len(node.parents) > 0:
        values = []
        for name in [node.name[0], node.name[2]]:
            for parent in node.parents:
                if name == parent.name:
                    value = state[nodes.index(parent)]
                    values.append(value)
        # find the probabilities given the state values
        dist = dist[values[0], values[1]]
    # return the probability given the node value
    p = dist[state[nodes.index(node)]]
    return p


def Gibbs_sampler(bayes_net, initial_state, evidence_map = {}):
    """Complete a single iteration of the Gibbs sampling algorithm 
    given a Bayesian network and an initial state value. 
    
    initial_state is a list of length 6 where: 
    index 0-2: represent skills of teams A,B,C (values lie in [0,3] inclusive)
    index 3-5: represent results of matches AvB, BvC, CvA (values lie in [0,2] inclusive)
    
    Returns the new state sampled from the probability distribution as a tuple of length 6.
    Return the sample as a tuple.    
    """
    nodes = list(bayes_net.nodes)
    initial_state = get_initial_state(nodes, initial_state)
    
    # get random node to change
    node = None
    while node is None or node in evidence_map:
        node = nodes[np.random.randint(len(nodes))]
    
    sample = list(initial_state)
    index = nodes.index(node)
    values = xrange(node.numValues)
    
    # get conditional probabilities distribution
    p_list = []
    for value in values:
        sample[index] = value
        p_list.append(get_markov_blanket_probability(nodes, node, sample))
    
    # normalize p_list
    p_list = np.array(p_list) / np.sum(p_list)
    
    # sample based on conditional probabilities
    value = np.random.choice(values, p=p_list)
    sample[index] = value

    return tuple(sample)

In [15]:
# arbitrary initial state for the game system
initial_state = [0,0,0,0,0,0]
sample = Gibbs_sampler(game_net, initial_state)
print sample

(1, 0, 0, 0, 0, 0)


2d: Metropolis-Hastings sampling
---
15 points

Now you will implement the Metropolis-Hastings algorithm, which is another method for estimating a probability distribution.
The general idea of MH is to build an approximation of a latent probability distribution by repeatedly generating a "candidate" value for each random variable in the system, and then probabilistically accepting or rejecting the candidate value based on an underlying acceptance function. Unlike Gibbs, in case of MH, the returned state can differ from the initial state at more than one variable.
This [cheat sheet](http://www.mit.edu/~ilkery/papers/MetropolisHastingsSampling.pdf) provides a nice intro.

This method method should just perform a single iteration of the algorithm. If an initial value is not given, default to a state chosen uniformly at random from the possible states. 

Note: DO NOT USE the given inference engines to run the sampling method, since the whole point of sampling is to calculate marginals without running inference. 


     "YOU WILL SCORE 0 POINTS IF YOU USE THE GIVEN INFERENCE ENGINES, OR ANY OTHER SAMPLING METHOD!!"
    

In [16]:
def MH_sampler(bayes_net, initial_state, evidence_map = {}):
    """Complete a single iteration of the MH sampling algorithm given a Bayesian network and an initial state value. 
    initial_state is a list of length 6 where: 
    index 0-2: represent skills of teams A,B,C (values lie in [0,3] inclusive)
    index 3-5: represent results of matches AvB, BvC, CvA (values lie in [0,2] inclusive)
    
    Returns the new state sampled from the probability distribution as a tuple of length 6. 
    """
    nodes = list(bayes_net.nodes)
    initial_state = get_initial_state(nodes, initial_state)
    
    sample = list(initial_state)
    candidate = list(initial_state)
    
    filtered_nodes = [node for node in nodes if node not in evidence_map]

    # propose a candidate
    for node in filtered_nodes:
        index = nodes.index(node)
        value = np.random.choice(node.numValues)
        candidate[index] = value
    
    # get acceptance probability
    current_probability = 1.
    candidate_probability = 1.
    for node in nodes:
        current_probability *= get_probability(nodes, node, initial_state)
        candidate_probability *= get_probability(nodes, node, candidate)
    
    if candidate_probability > current_probability:
        # If probability of new state is greater than that of the old state, you accept the candidate.
        sample = candidate
    else:
        # Else, you accept/reject the candidate by randomly choosing a value between 0 and 1 (excluding 0 and 1).
        alpha = candidate_probability / current_probability
        if alpha > np.random.uniform(0, 1):
            sample = candidate
    
    return tuple(sample)

In [17]:
# arbitrary initial state for the game system
initial_state = [0,0,0,0,0,0] 
sample = MH_sampler(game_net, initial_state)
print sample

(2, 1, 0, 0, 0, 1)


2e: Comparing sampling methods
----
20 points

Now we are ready for the moment of truth.

Given the same outcomes as in 2b, A beats B and A draws with C, you should now estimate the likelihood of different outcomes for the third match by running Gibbs sampling until it converges to a stationary distribution. 
We'll say that the sampler has converged when, for "N" successive iterations, the difference in expected outcome for the 3rd match differs from the previous estimated outcome by less than "delta". N is a positive integer. delta goes from (0,1). For the most stationary convergence, delta should be very small. N could typically take values like 10,20,...,100 or even more.

Measure how many iterations it takes for Gibbs and MH to converge to a stationary distribution over the posterior. See for yourself how close (or not) this stable distribution is to what the Inference Engine returned in 2b. And if not, try tuning those parameters(N and delta). (You might find the concept of "burn-in" period useful). 

For the purpose of this assignment, we'll say that the sampler has converged when, for 10 successive iterations, the difference in expected outcome for the third match differs from the previous estimated outcome by less than .001 (0.1%).

Repeat this experiment for Metropolis-Hastings sampling.

Which algorithm converges more quickly? By approximately what factor? For instance, if Metropolis-Hastings takes twice as many iterations to converge as Gibbs sampling, you'd say that it converged faster by a factor of 2. Fill in sampling_question() to answer both parts.


In [18]:
def get_posterior(sample_count):
    return np.array(sample_count, dtype=np.float32) / np.sum(sample_count)


def is_accepted(current_posterior, prev_posterior, delta):
    if prev_posterior is None:
        return False
    current_posterior = np.array(current_posterior)
    prev_posterior = np.array(prev_posterior)
    diff = np.array(np.abs(current_posterior - prev_posterior))
    return np.any(current_posterior != prev_posterior) and np.all(diff < delta)


def compare_sampling(bayes_net, initial_state, delta):
    """Compare Gibbs and Metropolis-Hastings sampling by calculating how long it takes for each method to converge."""    
    Gibbs_count = 0
    MH_count = 0
    MH_rejection_count = 0
    Gibbs_convergence = [0,0,0] # posterior distribution of the BvC match as produced by Gibbs 
    MH_convergence = [0,0,0] # posterior distribution of the BvC match as produced by MH
    # TODO: finish this function
    
    N = 100000
    BURN_IN = 10000
    CONSECUTIVE_THRESHOLD = 10
    nodes = list(bayes_net.nodes)
    AvB = bayes_net.get_node_by_name('AvB')
    BvC = bayes_net.get_node_by_name('BvC')
    CvA = bayes_net.get_node_by_name('CvA')
    
    BvC_index = nodes.index(BvC)
    evidence_map = { AvB: 0, CvA: 2 }
    
    initial_state = get_initial_state(nodes, initial_state)
    
    current_state = list(initial_state)
    for node, value in evidence_map.iteritems():
        current_state[nodes.index(node)] = value
    
    sample_count = [0, 0, 0]
    prev_posterior = None
    consecutive_iteration = 0
    for Gibbs_count in xrange(N):
        next_state = Gibbs_sampler(bayes_net, current_state, evidence_map)
        current_state = next_state
        sample_count[next_state[BvC_index]] += 1
        
        if Gibbs_count < BURN_IN:
            continue
        
        Gibbs_convergence = get_posterior(sample_count)
        
        if is_accepted(Gibbs_convergence, prev_posterior, delta):
            consecutive_iteration += 1
        else:
            consecutive_iteration = 0
        prev_posterior = Gibbs_convergence
        
        if consecutive_iteration >= CONSECUTIVE_THRESHOLD:
            break
    
    current_state = list(initial_state)
    for node, value in evidence_map.iteritems():
        current_state[nodes.index(node)] = value
    
    sample_count = [0, 0, 0]
    prev_posterior = None
    consecutive_iteration = 0
    for MH_count in xrange(N):
        next_state = MH_sampler(bayes_net, current_state, evidence_map)
        
        if np.all(current_state == next_state):
            MH_rejection_count += 1
            
        current_state = next_state
        sample_count[next_state[BvC_index]] += 1
        
        if MH_count < BURN_IN:
            continue
        
        MH_convergence = get_posterior(sample_count)
        
        if is_accepted(MH_convergence, prev_posterior, delta):
            consecutive_iteration += 1
        else:
            consecutive_iteration = 0
        prev_posterior = MH_convergence
        
        if consecutive_iteration >= CONSECUTIVE_THRESHOLD:
            break
    
    return list(Gibbs_convergence), list(MH_convergence), Gibbs_count, MH_count, MH_rejection_count

In [19]:
initial_state = [0,0,0,0,0,0]
delta = 0.001
compare_sampling(game_net,initial_state, delta)

([0.24962541, 0.41614226, 0.33423233],
 [0.26151234, 0.42123663, 0.31725103],
 10010,
 10010,
 7968)

In [20]:
def sampling_question():
    """Question about sampling performance."""
    # TODO: assign value to choice and factor
    choice = 0
    options = ['Gibbs','Metropolis-Hastings']
    factor = 1
    return options[choice], factor

And we are done!